In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def csv2df(pattern):
    df = None
    for f in sorted(glob.iglob(pattern)):
        df0 = pd.read_csv(f, header=None, names=["test", "info", "time"])
        if df is None:
            df = df0
        else:
            df["time"] = np.minimum(df["time"], df0["time"])
    df[["ds", "name"]] = df["test"].str.extract(r"([^_]+)_(.*)_test")
    return df[["ds", "name", "info", "time"]]

tdf = None
mdf = None
for s in ["win", "lnx"]:
    df = csv2df("*-%s-*.csv" % s)
    tdf0 = df.loc[df["name"] != "memtrack"][["ds", "name", "time"]].rename(columns={"time": s})
    tdf0 = tdf0.set_index(['ds', 'name'])
    tdf0[s] = tdf0.groupby('name')[s].transform(lambda x: x / x.loc['imap'])
    tdf0 = tdf0.reset_index()
    if tdf is None:
        tdf = tdf0
    else:
        tdf = tdf.merge(tdf0, how="left")
    mdf0 = df.loc[df["name"] == "memtrack"][["ds", "name", "info"]].rename(columns={"info": s})
    mdf0[s] = mdf0[s].apply(lambda x: int(x.split("/")[1]) if "/" in x else int(x))
    if mdf is None:
        mdf = mdf0
    else:
        mdf = mdf.merge(mdf0, how="left")

In [2]:
for t, gdf in tdf.groupby("name", sort=False):
    gdf = gdf.set_index("ds")
    plt.figure(figsize=(6,3), dpi=100, facecolor="white")
    plt.title(t)
    gdf.plot.barh(ax=plt.gca()).invert_yaxis()
    plt.gca().set_xlim(0,22)
    plt.gca().set(ylabel=None)
    for container in plt.gca().containers:
        plt.gca().bar_label(container, fmt="%0.2f", padding=4.0, fontsize="xx-small")
    plt.savefig(t + ".png")
    #plt.show()
    plt.close()

In [3]:
for t, gdf in mdf.groupby("name", sort=False):
    gdf = gdf.set_index("ds")
    plt.figure(figsize=(6,3), dpi=100, facecolor="white")
    plt.title(t)
    gdf.plot.barh(ax=plt.gca()).invert_yaxis()
    plt.gca().set(ylabel=None)
    for container in plt.gca().containers:
        plt.gca().bar_label(container, fmt="%0.0f", padding=4.0, fontsize="xx-small")
    plt.savefig(t + ".png")
    #plt.show()
    plt.close()